# AI Agent with LangGraph and ReAct Pattern
---

This project demonstrates the creation of an AI agent from scratch.

## Contributing

Contributions are welcome! Please follow these steps:

1. Fork the repository.
2. Create a new branch: `git checkout -b feature-name`.
3. Commit your changes: `git commit -m 'Add feature'`.
4. Push to the branch: `git push origin feature-name`.
5. Open a pull request.

## License

This project is licensed under the [MIT License](LICENSE).

---



### Step 1: Setting up the environment

In [51]:
# install dependencies if they do not exist
# !pip install openai regex httpx python-dotenv

In [52]:
import openai                                                                # Imports the legacy OpenAI SDK (used in versions <1.0 for direct API calls)
import re                                                                    # Imports the regular expressions module for pattern matching and text processing
import httpx                                                                 # Imports the HTTPX library for async-capable HTTP requests
import os                                                                    # Imports the OS module to interact with environment variables and file paths
from dotenv import load_dotenv                                               # Imports the function to load environment variables from a .env file

_ = load_dotenv()                                                            # Loads environment variables from a .env file into the runtime environment
from openai import OpenAI                                                    # Imports the OpenAI client class (used in SDK v1.0+ for client-based API access)

In [53]:
client = OpenAI()                                                             # Initializes the OpenAI client using the API key from the "OPENAI_API_KEY" environment variable

In [54]:
chat_completion = client.chat.completions.create(                            # Sends a chat completion request to the OpenAI API using the client instance
    model="gpt-3.5-turbo",                                                   # Specifies the language model to use (in this case, GPT-3.5 Turbo)
    messages=[                                                               # Provides a list of messages representing the conversation history
        {"role": "user", "content": "Hello world"}                           # Defines a single message where the user says "Hello world"
    ]
)

In [55]:
chat_completion.choices[0].message.content                                   # Retrieves the assistant's text response from the first result in the list of generated choices returned by the OpenAI chat completion API

'Hello! How can I assist you today?'

In [56]:
class Agent:
    def __init__(self, system=""):                                           # Initializes the Agent with an optional system prompt
        self.system = system                                                 # Stores the system message (if any)
        self.messages = []                                                   # Initializes an empty message history list
        if self.system:                                                      # If a system prompt is provided...
            self.messages.append({"role": "system", "content": system})      # ...add it as the first message

    def __call__(self, message):                                             # Makes the Agent instance callable like a function, taking a user message
        self.messages.append({"role": "user", "content": message})           # Adds the user's message to the history
        result = self.execute()                                              # Calls the execute method to get the assistant's reply
        self.messages.append({"role": "assistant", "content": result})       # Appends the assistant's response to the history
        return result                                                        # Returns the assistant's response

    def execute(self):                                                       # Defines how the agent communicates with the OpenAI API
        completion = client.chat.completions.create(                         # Sends all messages to the chat API using GPT-4o
                        model="gpt-4o",                                      # Specifies the GPT-4 Omni model
                        temperature=0,                                       # Sets deterministic output (no randomness)
                        messages=self.messages)                              # Sends the full conversation history
        return completion.choices[0].message.content                         # Returns the assistant's message from the first choice

In [57]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

In [58]:
def calculate(what):                                                      # Defines a function that evaluates a mathematical expression passed as a string
    return eval(what)                                                     # Uses Python's eval() to compute the result (⚠️ unsafe with untrusted input)

def average_dog_weight(name):                                             # Returns average weight information based on the dog's breed name
    if name in "Scottish Terrier":                                        # Checks if input is a substring of "Scottish Terrier" (not exact match)
        return("Scottish Terriers average 20 lbs")                        # Returns weight for Scottish Terrier
    elif name in "Border Collie":                                         # Checks if input is a substring of "Border Collie"
        return("a Border Collies average weight is 37 lbs")               # Returns weight for Border Collie
    elif name in "Toy Poodle":                                            # Checks if input is a substring of "Toy Poodle"
        return("a toy poodles average weight is 7 lbs")                   # Returns weight for Toy Poodle
    else:
        return("An average dog weights 50 lbs")                           # Default fallback for unknown breeds

known_actions = {                                                         # Defines a dictionary mapping action names (as strings) to their corresponding functions
    "calculate": calculate,                                               # Maps "calculate" to the calculate() function
    "average_dog_weight": average_dog_weight                              # Maps "average_dog_weight" to the average_dog_weight() function
}

In [59]:
abot = Agent(prompt)                                                      # Creates an instance of the Agent class using the provided system prompt as context for the assistant

In [60]:
result = abot("How much does a toy poodle weigh?")                        # Sends the user's question to the Agent, which queries the model and returns a response
print(result)                                                             # Prints the assistant's reply (e.g., from GPT-4o based on the conversation history)

Thought: I should look up the average weight of a Toy Poodle using the average_dog_weight action.
Action: average_dog_weight: Toy Poodle
PAUSE


In [61]:
result = average_dog_weight("Toy Poodle")                                 # Calls the average_dog_weight function with "Toy Poodle" as input and stores the returned weight description in 'result'

In [62]:
result                                                                    # A variable that holds the output returned by the average_dog_weight function for the given dog breed input

'a toy poodles average weight is 7 lbs'

In [63]:
next_prompt = "Observation: {}".format(result)                            # Formats a string by inserting the value of 'result' into an observation statement, useful for reasoning chains or agent memory

In [64]:
abot(next_prompt)                                                         # Sends the formatted observation string to the Agent, which appends it to the conversation history and gets a response from the language model

'Answer: A Toy Poodle weighs an average of 7 lbs.'

In [65]:
abot.messages                                                             # Displays the full conversation history stored in the Agent, including system prompts, user messages, assistant replies, and any intermediate observations

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'},
 {'role': 'user', 'content': 'How much does a 

In [66]:
abot = Agent(prompt)                                                    # Instantiates the Agent with the initial system prompt, setting the context or behavior guidelines for the language model

In [67]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""                                        # Defines a multiline string containing the user's question about two dog breeds

abot(question)                                                          # Sends the question to the Agent, which adds it to the message history, queries the model, and returns the assistant's calculated or inferred response

'Thought: I need to find the average weight of both a Border Collie and a Scottish Terrier, then add them together to find the combined weight.\nAction: average_dog_weight: Border Collie\nPAUSE'

In [68]:
next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))  # Inserts the returned string from average_dog_weight("Border Collie") into an observation statement
print(next_prompt)                                                           # Outputs: Observation: a Border Collies average weight is 37 lbs

Observation: a Border Collies average weight is 37 lbs


In [69]:
abot(next_prompt)                                                               # Sends the observation (e.g., about a Border Collie's weight) to the Agent, which updates the message history and retrieves a contextual response from the model

'Action: average_dog_weight: Scottish Terrier\nPAUSE'

In [70]:
next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))  # Formats the observation string using the weight info for a Scottish Terrier
print(next_prompt)                                                              # Outputs: Observation: Scottish Terriers average 20 lbs

Observation: Scottish Terriers average 20 lbs


In [71]:
abot(next_prompt)                                                               # Sends the new observation about the Scottish Terrier's weight to the Agent, which logs it in the conversation and gets an updated response from the language model

'Thought: Now that I have the average weights of both dogs, I can calculate their combined weight by adding the two values together.\nAction: calculate: 37 + 20\nPAUSE'

In [72]:
next_prompt = "Observation: {}".format(eval("37 + 20"))                         # Evaluates the expression "37 + 20" to get 57, then formats it as: "Observation: 57"
print(next_prompt)                                                              # Outputs: Observation: 57

Observation: 57


In [73]:
abot(next_prompt)                                                               # Sends the numeric observation ("Observation: 57") to the Agent, allowing the assistant to reason further or acknowledge the calculated combined weight in the conversation context

'Answer: The combined weight of a Border Collie and a Scottish Terrier is 57 lbs.'

### Add loop 

In [74]:
action_re = re.compile('^Action: (\w+): (.*)$')                                 # python regular expression to selection action

In [75]:
def query(question, max_turns=5):                                               # Defines a function to interact with the agent and execute tool calls for a fixed number of steps
    i = 0                                                                       # Initializes a counter to track the number of interaction turns
    bot = Agent(prompt)                                                         # Creates a new agent instance using the provided system prompt
    next_prompt = question                                                      # Sets the initial prompt to the user’s question

    while i < max_turns:                                                        # Loops for a maximum of `max_turns` iterations
        i += 1                                                                  # Increments the turn counter
        result = bot(next_prompt)                                               # Sends the current prompt to the agent and gets a response
        print(result)                                                           # Prints the assistant's response

        # Looks for lines in the response that match the expected action format (e.g., Action: function("input"))
        actions = [
            action_re.match(a) 
            for a in result.split('\n') 
            if action_re.match(a)
        ]

        if actions:                                                            # If the model suggested an action
            action, action_input = actions[0].groups()                         # Extract the action name and its input argument

            if action not in known_actions:                                    # If the action isn't in the predefined tools
                raise Exception("Unknown action: {}: {}".format(action, action_input))  # Raise an error

            print(" -- running {} {}".format(action, action_input))            # Logs the action being executed
            observation = known_actions[action](action_input)                  # Executes the tool function and stores the result
            print("Observation:", observation)                                 # Prints the observation returned from the tool
            next_prompt = "Observation: {}".format(observation)                # Formats the observation as the next input to the agent
        else:
            return                                                             # Stops if no further action is suggested by the assistant

In [76]:
question = """I have 2 dogs, a border collie and a scottish terrier. \  # Defines a multi-line string asking for the combined weight of two specific dog breeds
What is their combined weight"""                                               # Continues the question on the next line due to the backslash

query(question)                                                  # Calls the query() function with the user's question, initiating a multi-turn reasoning loop where the agent may use tool functions to compute an answer

Thought: I need to find the average weight of both a Border Collie and a Scottish Terrier, then add them together to find the combined weight.
Action: average_dog_weight: Border Collie
PAUSE
 -- running average_dog_weight Border Collie
Observation: a Border Collies average weight is 37 lbs
Action: average_dog_weight: Scottish Terrier
PAUSE
 -- running average_dog_weight Scottish Terrier
Observation: Scottish Terriers average 20 lbs
Thought: Now that I have the average weights of both the Border Collie and the Scottish Terrier, I can calculate their combined weight.
Action: calculate: 37 + 20
PAUSE
 -- running calculate 37 + 20
Observation: 57
Answer: The combined average weight of a Border Collie and a Scottish Terrier is 57 lbs.
